In [ ]:
import warnings
warnings.filterwarnings('ignore')


# Where ML Fits into Causal Inference (review)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Mixtape-Sessions/Machine-Learning/blob/main/Labs/python/Causal%20via%20Prediction.ipynb)

The traditional go-to tool for causal inference is multiple regression:
$$
Y_i = \delta D_i + X_i'\beta+\varepsilon_i,
$$
where $D_i$ is the "treatment" or causal variable whose effects we are interested in, and $X_i$ is a vector of controls, conditional on which we are willing to assume $D_i$ is as good as randomly assigned.


> *example:* Suppose we are interested in the magnitude of racial discrimination in the labor market. One way to conceptualize this is the difference in earnings between two workers who are identical in productivity, but differ in their race, or, the "effect" of race. Then $D_i$ would be an indicator for, say, a Black worker. $Y_i$ would be earnings, and $X_i$ would be characteristics that capture determinants of productivity, including educational attainment, cognitive ability, and other background characteristics.

Where does machine learning fit into causal inference? It might be tempting to treat
this regression as a prediction exercise where we are predicting $Y_{i}$
given $D_{i}$ and $X_{i}$. Don't give in to this temptation. We are not
after a prediction for $Y_{i}$, we are after a coefficient on $D_{i}$.
Modern machine learning algorithms are finely tuned for producing
predictions, but along the way they compromise coefficients. So how can we
deploy machine learning in the service of estimating the causal coefficient $\delta $?

To see where ML fits in, first remember that an equivalent way to estimate $%
\delta $ is the following three-step procedure:


1.   Regress $Y_{i}$ on $X_{i}$ and compute the residuals, $\tilde{Y}%
_{i}=Y_{i}-\hat{Y}_{i}^{OLS}$, where $\hat{Y}_{i}^{OLS}=X_{i}^{\prime
}\left( X^{\prime }X\right) ^{-1}X^{\prime }Y$
2.   Regress $D_{i}$ on $X_{i}$ and compute the residuals, $\tilde{D}%
_{i}=D_{i}-\hat{D}_{i}^{OLS}$, where $\hat{D}_{i}^{OLS}=X_{i}^{\prime
}\left( X^{\prime }X\right) ^{-1}X^{\prime }D$

3. Regress $\tilde{Y}_{i}$ on $\tilde{D}_{i}$.

Steps 1 and 2 are prediction exercises--ML's wheelhouse. When OLS isn't the right tool for the job, we can replace OLS in those steps with machine learning:

1.   Predict $Y_{i}$ based on $X_{i}$ using ML and compute the residuals, $\tilde{Y}%
_{i}=Y_{i}-\hat{Y}_{i}^{ML}$, where $\hat{Y}_{i}^{ML}$ is the prediction from an ML algorithm
2.   Predict $D_{i}$ based on $X_{i}$ using ML and compute the residuals, $\tilde{D}%
_{i}=D_{i}-\hat{D}_{i}^{ML}$, where $\hat{D}_{i}^{ML}$ is the prediction from an ML algorithm

3. Regress $\tilde{Y}_{i}$ on $\tilde{D}_{i}$.

This is the basis for the two major methods we'll look at today: The first is "Post-Double Selection Lasso" (Belloni, Chernozhukov, Hansen). The second is "Double-Debiased Machine Learning" (Chernozhukov, Chetverikov, Demirer, Duflo, Hansen, Newey, Robins)

# Post Double Selection Lasso (PDS Lasso)

## Load useful packages:
pandas, numpy, linear_model (from sklearn), and KFold (from sklearn.model_selection)

Try it yourself first

### Cheat

In [ ]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import KFold


## Read in data and have a look at it
We'll use the NLSY data from yesterday

Try it yourself

### Cheat

In [ ]:
nlsy=pd.read_csv('https://github.com/Mixtape-Sessions/Machine-Learning/blob/main/Labs/data/nlsy97.csv?raw=true')
nlsy


## Define outcome, regressor of interest
y = lnw_2016

d = black

Try it yourself:

### Cheat

In [ ]:
y=nlsy['lnw_2016']
d=nlsy[['black']]


## Simple Regression with no Controls
Regress y on d and print out coefficient
Try it yourself

In [ ]:
# instantiate and fit a linear regression object

# print out regression coefficient


### Cheat

In [ ]:
lm=linear_model.LinearRegression().fit(d,y)
print("Simple regression race gap: {:.3f}".format(lm.coef_[0]))


### ...
Is this the effect we're looking for?

Let's try a regression where we control for a few things: education (linearly), experience (linearly), and cognitive ability (afqt, linearly).

Try it yourself!

In [ ]:
# define X, matrix of the d and the controls we want

# run regression

# print out coefficient


### Cheat

In [ ]:
# define RHS, matrix of the d and the controls we want
RHS=nlsy[['black','educ','exp','afqt']]
# run regression
lm.fit(RHS,y)
# print out coefficient
print("Multiple regression-adjusted race gap: {:.3f}".format(lm.coef_[0]))




###...
How does it compare to the simple regression?

But who is to say the controls we included are sufficient? We have a whole host (hundred!) of other potential controls, not to mention that perhaps the controls we did put in enter linearly. This is a job for ML!

To prep, let's define a matrix X with all of our potential controls:

In [ ]:
X=nlsy.drop(columns=['lnw_2016','black'])


## Post Double Selection Lasso

### Step 1: Lasso the outcome on X
Try it yourself. Don't forget to standardize Xs

#### Cheat

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)
X = pd.DataFrame(data=scaler.transform(X),columns = X.columns)

lassoy = linear_model.LassoCV(max_iter=1000).fit(X, y)


### Step 2: Lasso the treatment on X
Try it yourself

#### Cheat

In [ ]:
lassod = linear_model.LassoCV(max_iter=1000).fit(X, d)


### Step 3: Form the union of controls
Try it yourself

#### Cheat

In [ ]:
Xunion=X.iloc[:,(lassod.coef_!=0) + (lassoy.coef_!=0)]
Xunion.head()


### Concatenate treatment with union of controls and regress y on that and print out estimate
Try yourself

#### Cheat

In [ ]:
rhs=pd.concat([d,Xunion],axis=1)
fullreg=linear_model.LinearRegression().fit(rhs,y)
print("PDS regression earnings race gap: {:.3f}".format(fullreg.coef_[0]))


## Double-Debiased Machine Learning
For simplicity, we will first do it without sample splitting

### Step 1: Ridge outcome on Xs, get residuals
Try yourself

#### Cheat

In [ ]:
ridgey = linear_model.RidgeCV().fit(X, y)
yresid=y-ridgey.predict(X)


### Step 2: Ridge treatment on Xs, get residuals
Try yourself

#### Cheat

In [ ]:
ridged = linear_model.RidgeCV().fit(X, d)
dresid=d-ridged.predict(X)


### Step 3: Regress y resids on d resids and print out estimate
Try yourself

####Cheat

In [ ]:
dmlreg=linear_model.LinearRegression().fit(dresid,yresid)
print("DML regression earnings race gap: {:.3f}".format(dmlreg.coef_[0]))


### The real thing: with sample splitting

In [ ]:
# create our sample splitting "object"
kf = KFold(n_splits=5,shuffle=True,random_state=42)

# apply the splits to our Xs
kf.get_n_splits(X)

# initialize columns for residuals
yresid = y*0
dresid = d*0

# Now loop through each fold
for train_index, test_index in kf.split(X):
  X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]
  d_train, d_test = d.iloc[train_index,:], d.iloc[test_index,:]

  # Do DML thing
  # Ridge y on training folds:
  ridgey.fit(X_train, y_train)

  # but get residuals in test set
  yresid.iloc[test_index]=y_test-ridgey.predict(X_test)

  #Ridge d on training folds
  ridged.fit(X_train, d_train)
  #but get residuals in test set
  dresid.iloc[test_index,:]=d_test-ridged.predict(X_test)


# Regress resids
dmlreg=linear_model.LinearRegression().fit(dresid,yresid)

print("DML regression earnings race gap: {:.3f}".format(dmlreg.coef_[0]))


You want standard errors, do you?

In [ ]:
import statsmodels.api as sm
rhs = sm.add_constant(dresid)
model = sm.OLS(yresid, rhs)
results = model.fit(cov_type='HC3')
print(results.summary())


## Now do DML using Random Forest!

### Cheat

In [ ]:
# import random forest
from sklearn.ensemble import RandomForestRegressor
# instantiate random forest objects
rfy=RandomForestRegressor(n_estimators=100)
rfd = rfy

# create our sample splitting "object"
kf = KFold(n_splits=5,shuffle=True,random_state=42)

# apply the splits to our Xs
kf.get_n_splits(X)


# initialize columns for residuals
yresidrf = y*0
dresidrf = d*0

# Now loop through each fold
for train_index, test_index in kf.split(X):
  X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]
  d_train, d_test = d.iloc[train_index,:], d.iloc[test_index,:]

  # Do DML thing
  # Ridge y on training folds:
  rfy.fit(X_train, y_train)

  # but get residuals in test set
  yresidrf.iloc[test_index]=y_test-rfy.predict(X_test)

  #Ridge d on training folds
  rfd.fit(X_train, d_train)
  #but get residuals in test set
  dresidrf.iloc[test_index,:]=d_test-rfd.predict(X_test).reshape(-1,1)


# Regress resids
dmlreg=linear_model.LinearRegression().fit(dresidrf,yresidrf)

print("DML (Random forest) regression earnings race gap: {:.3f}".format(dmlreg.coef_[0]))
rhs = sm.add_constant(dresidrf)
model = sm.OLS(yresidrf, rhs)
results = model.fit(cov_type='HC3')
print(results.summary())
